# Classificação e contagem de Salgadinhos com Deep Learning
Autores: Vicente Knobel Borges e Luiz Gustavo Xavier

Neste relatório é apresentada a solução de um problema de visão computacional aplicado a indústria de alimentos, como requisitos para a disciplina INE410121 - Visão Computacional. O projeto consiste na aplicação de algoritmos clássicos de visão computacional e redes neurais artificiais para solução do problema de contagem de salgadinhos sortidos em caixas mistas. Em nossa melhor solução com o *fine tune* do modelo pré-treinado [RF-DETR](https://github.com/roboflow/rf-detr), foi possível identificar corretamente diversas classes de salgadinhos em fotografias diversas de caixas, obtendo YY de acurácia em nosso dataset.

O processo de geração do modelo se deu por uma sequencia de passos:
1. Busca manual de imagens exemplo no google imagens
2. Geração de [*novel view synthesis*](https://en.wikipedia.org/wiki/View_synthesis) das imagens adquiridas usando o modelo [Nano Banana Pro](https://blog.google/technology/ai/nano-banana-pro/) da Google.
3. *Labeling* bruto usando [sam3](https://github.com/facebookresearch/sam3)
4. Refinamento das labels usando [Label Studio](https://labelstud.io/)
5. Treinamento usando modelos [RF-DETR](https://github.com/roboflow/rf-detr) 

## Geração do *dataset*

### Busca manual pelo google imagens
Começamos o processo definindo quais salgadinhos o modelo deve indentificar. Buscamos representar os salgadinhos mais comuns vendidos por empresas de alimentos para festas, chegando na lista:
- Bolinha de queijo
- Canapé
- Canudo
- Coxinha
- Croquete
- Empadinha
- Enroladinho de salsicha
- Esfiha
- Folhado
- Pastelzinho
- Pão de queijo
- Quibe
- Risoles
- Sanduiche

Buscamos 30 imagens de cada classe, além de um conjunto de imagens de bandejas cheias, denominado 'grupo'. A Heuristica de inclusão priorizava a escolha de imagens por:
- Conter multiplos exemplares do salgadinho buscado em uma unica foto
- Salgadinhos fotografados no angulo *top-down*
- Salgadinhos em bandejas são preferenciais a salgadinhos em potes ou empilhados

![imagens de coxinhas selecionadas](relatorio2Images/coxinhas_google.jpg)
Acima: Imagens de coxinhas selecionadas

### Novel view synthesis com Nano Banana Pro
No processo de buscar exemplos no google, nos deparamos com um forte viés em fotos de uma perspectiva lateral ao corpo de salgados, coerente com propagandas e divulgação. Embora seja relevante que nosso modelo trabalhe com este outro angulo de visão, achamos preocupante a ausência de fotos de vista superior.

A vista superior é a perspectiva que estipulamos como nosso caso de uso principal com o modelo sendo colocado em produção por padeiros interessados em contar os salgados em suas caixas antes de despachar para entrega.

Para gerar essas vistas fizemos uso do novo modelo de geração, edição e manipulação de imagens da Google, o [Nano Banana Pro](https://blog.google/technology/ai/nano-banana-pro/).

O modelo está disponivel pelo [serviço de API dos modelos Gemini](https://ai.google.dev/gemini-api/docs/image-generation) e por meio de *prompt engineering*, implementamos uma geração que extrapola quatro novas imagens a partir da imagem original, em uma unica geração.

- Uma geração de vista lateral, similar a imagem original;
- Uma geração top-down, dos salgadinhos no chão - mudando o fundo dos salgadinhos e variando o fundo;
- Uma geração top-down, com luz noturna, diversificando condições de iluminação
- Uma geração top-down dos salgados na sua disposição original.

![imagens de coxinhas geradas pelo modelo Nano Banana](relatorio2Images/nanoBanana.jpg)
Acima: Imagens de coxinhas geradas pelo modelo Nano Banana

#### Novel View Synthesis: código

##### bibliotecas

In [ ]:
!pip install -U google-genai #api para acesso aos modelos de IA da Google
!pip install -U pillow #bibilioteca de manipulação de imagens
!pip install -U opencv-python #bibilioteca de visão computacional
!pip install -U tqdm #progress bar

##### imports e definições

In [ ]:
api_key='' #your google api key here

from google import genai
from google.genai import types
from PIL import Image

import os
from pathlib import Path
from PIL import Image

from tqdm import tqdm

import cv2
import numpy as np
import glob
import math
from pathlib import Path

##### funções

In [ ]:
def nanoBananaGeneration(client, prompt, image_path_in, image_path_out):
    # Define a proporção de aspecto da imagem gerada
    aspect_ratio = "5:4" # "1:1","2:3","3:2","3:4","4:3","4:5","5:4","9:16","16:9","21:9"
    # Define a resolução da imagem gerada
    resolution = "1K" # "1K", "2K", "4K"
    
    # Faz uma chamada ao modelo Gemini para gerar conteúdo (texto e imagem)
    response = client.models.generate_content(
        model="gemini-3-pro-image-preview",
        # Passa o prompt de texto e a imagem de entrada como conteúdo
        contents=[
            prompt,
            Image.open(image_path_in),
        ],
        # Configura os parâmetros de geração
        config=types.GenerateContentConfig(
            # Define que a resposta pode conter texto e imagem
            response_modalities=['TEXT', 'IMAGE'],
            # Configura as propriedades da imagem a ser gerada
            image_config=types.ImageConfig(
                aspect_ratio=aspect_ratio,
                image_size=resolution
            ),
        )
    )
    
    # Itera sobre as partes da resposta gerada
    for part in response.parts:
        # Se a parte contém texto, imprime no console
        if part.text is not None:
            print(part.text)
        # Se a parte contém uma imagem, salva no caminho de saída
        elif image:= part.as_image():
            image.save(image_path_out)

def collect_image_paths(input_folder):
    # Define as extensões de arquivo de imagem aceitas
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
    # Inicializa lista vazia para armazenar os caminhos
    paths = []
    # Percorre recursivamente todos os arquivos e subpastas da pasta de entrada
    for root, _, files in os.walk(input_folder):
        # Para cada arquivo encontrado
        for f in files:
            # Verifica se a extensão do arquivo está na lista de extensões aceitas (ignorando maiúsculas/minúsculas)
            if Path(f).suffix.lower() in exts:
                # Adiciona o caminho completo do arquivo à lista
                paths.append(Path(root) / f)
    # Retorna a lista de caminhos das imagens encontradas
    return paths

def filter_unprocessed(paths, input_folder, output_folder):
    # Converte as pastas de entrada e saída para objetos Path
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    # Inicializa lista para armazenar apenas os caminhos não processados
    filtered = []
    # Para cada caminho de imagem de origem
    for src_path in paths:
        # Obtém o caminho relativo da imagem em relação à pasta de entrada
        rel = src_path.relative_to(input_folder)
        # Remove a extensão do caminho relativo
        rel_no_ext = rel.with_suffix("")
        # Constrói o caminho de destino na pasta de saída com extensão .jpg
        dst_path = (output_folder / rel_no_ext).with_suffix(".jpg")
        # Se o arquivo de destino não existe, adiciona à lista de não processados
        if not dst_path.exists():
            filtered.append(src_path)
    # Retorna apenas os caminhos que ainda não foram processados
    return filtered

def process_paths(paths, input_folder, output_folder, client, prompt):
    # Converte as pastas de entrada e saída para objetos Path
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    
    # Garante que a pasta de saída raiz existe (cria se não existir)
    output_folder.mkdir(parents=True, exist_ok=True)
    # Itera sobre os caminhos com barra de progresso usando tqdm
    for src_path in tqdm(paths, desc="Processing images"):
        # Obtém o caminho relativo da imagem em relação à pasta de entrada
        rel_path = src_path.relative_to(input_folder)
        # Remove a extensão do caminho relativo
        rel_no_ext = rel_path.with_suffix("")
        # Constrói o caminho de destino na pasta de saída com extensão .jpg
        dst_path = (output_folder / rel_no_ext).with_suffix(".jpg")
        # Garante que as subpastas necessárias existem no destino
        dst_path.parent.mkdir(parents=True, exist_ok=True)
        # Tenta processar a imagem
        try:
            # Abre a imagem de origem e converte para RGB
            img = Image.open(src_path).convert("RGB")
            # Chama a função de geração com o cliente, prompt e caminhos
            nanoBananaGeneration(client, prompt, src_path, dst_path)
        # Se houver qualquer erro, ignora e continua para a próxima imagem
        except:
            continue

def resize_to_box(img, box_w=1440, box_h=720):
    # Obtém a altura e largura da imagem
    h, w = img.shape[:2]
    # Calcula o fator de escala para caber na caixa, mantendo proporção (usa o menor dos dois)
    scale = min(box_w / w, box_h / h)
    # Calcula a nova largura aplicando o fator de escala
    new_w = int(w * scale)
    # Calcula a nova altura aplicando o fator de escala
    new_h = int(h * scale)
    # Redimensiona a imagem com interpolação INTER_AREA (boa para redução)
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def create_collage(input_folder, out_path="collage.jpg", box_w=1440, box_h=720):
    # Converte a pasta de entrada para objeto Path
    input_folder = Path(input_folder)
    # Obtém lista ordenada de todos os caminhos de arquivos na pasta
    image_paths = sorted(glob.glob(str(input_folder / "*")))
    # Inicializa lista vazia para armazenar as imagens carregadas
    images = []
    # Para cada caminho de imagem
    for p in image_paths:
        # Tenta carregar a imagem usando OpenCV
        img = cv2.imread(p)
        # Se a imagem não pôde ser carregada, pula para a próxima
        if img is None:
            continue
        # Redimensiona a imagem para caber na caixa definida
        img = resize_to_box(img, box_w, box_h)
        # Adiciona a imagem redimensionada à lista
        images.append(img)
    # Se nenhuma imagem foi carregada, imprime mensagem e sai
    if not images:
        print("No images found.")
        return
    # --- constrói grade automaticamente ---
    # Conta o número total de imagens
    n = len(images)
    # Calcula o número de colunas (arredonda para cima a raiz quadrada)
    cols = math.ceil(math.sqrt(n))
    # Calcula o número de linhas necessárias
    rows = math.ceil(n / cols)
    # Define largura e altura de cada célula da grade
    cell_w, cell_h = box_w, box_h
    # Calcula largura total da colagem
    collage_w = cols * cell_w
    # Calcula altura total da colagem
    collage_h = rows * cell_h
    # Cria um array numpy vazio (preto) para a colagem completa
    collage = np.zeros((collage_h, collage_w, 3), dtype=np.uint8)
    # Inicializa índice da imagem atual
    i = 0
    # Para cada linha da grade
    for r in range(rows):
        # Para cada coluna da grade
        for c in range(cols):
            # Se já processou todas as imagens, interrompe
            if i >= n: break
            # Pega a imagem atual
            img = images[i]
            # Calcula o deslocamento vertical para centralizar a imagem na célula
            y_offset = r * cell_h + (cell_h - img.shape[0]) // 2
            # Calcula o deslocamento horizontal para centralizar a imagem na célula
            x_offset = c * cell_w + (cell_w - img.shape[1]) // 2
            # Coloca a imagem na posição calculada dentro da colagem
            collage[y_offset:y_offset+img.shape[0],
                    x_offset:x_offset+img.shape[1]] = img
            # Avança para a próxima imagem
            i += 1

    # Redimensiona a colagem para ter o tamanho final da caixa de interesse
    collage = resize_to_box(collage, box_w, box_h)
    # Salva a colagem final no caminho especificado
    cv2.imwrite(out_path, collage)
    # Imprime mensagem confirmando o salvamento
    print("Saved:", out_path)

def slice_images_into_four(input_folder, output_folder="imagens_cortadas"):
    # Converte as pastas de entrada e saída para objetos Path
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    
    # Define as extensões de arquivo de imagem aceitas
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
    
    # Garante que a pasta de saída raiz existe (cria se não existir)
    output_folder.mkdir(parents=True, exist_ok=True)
    
    # Coleta todos os caminhos de imagens recursivamente
    image_paths = []
    for root, _, files in os.walk(input_folder):
        for f in files:
            if Path(f).suffix.lower() in exts:
                image_paths.append(Path(root) / f)
    
    # Itera sobre os caminhos com barra de progresso usando tqdm
    for src_path in tqdm(image_paths, desc="Slicing images"):
        # Tenta processar a imagem
        try:
            # Carrega a imagem usando OpenCV
            img = cv2.imread(str(src_path))
            # Se a imagem não pôde ser carregada, pula para a próxima
            if img is None:
                continue
            
            # Obtém a altura e largura da imagem
            h, w = img.shape[:2]
            # Calcula a altura de cada fatia (metade da altura total)
            half_h = h // 2
            # Calcula a largura de cada fatia (metade da largura total)
            half_w = w // 2
            
            # Define as 4 fatias da imagem (superior esquerda, superior direita, inferior esquerda, inferior direita)
            slices = [
                img[0:half_h, 0:half_w],           # fatia 0: superior esquerda
                img[0:half_h, half_w:w],           # fatia 1: superior direita
                img[half_h:h, 0:half_w],           # fatia 2: inferior esquerda
                img[half_h:h, half_w:w]            # fatia 3: inferior direita
            ]
            
            # Obtém o caminho relativo da imagem em relação à pasta de entrada
            rel_path = src_path.relative_to(input_folder)
            # Obtém o caminho relativo sem a extensão
            rel_no_ext = rel_path.with_suffix("")
            
            # Para cada fatia (0 a 3)
            for idx, slice_img in enumerate(slices):
                # Constrói o caminho de destino adicionando o índice da fatia ao nome
                dst_path = (output_folder / f"{rel_no_ext}_{idx}").with_suffix(".jpg")
                # Garante que as subpastas necessárias existem no destino
                dst_path.parent.mkdir(parents=True, exist_ok=True)
                # Salva a fatia da imagem
                cv2.imwrite(str(dst_path), slice_img)
        
        # Se houver qualquer erro, ignora e continua para a próxima imagem
        except Exception as e:
            print(f"Error processing {src_path}: {e}")
            continue
    
    # Imprime mensagem de conclusão
    print(f"Slicing complete! Images saved to: {output_folder}")



##### execução
Folder com as imagens baixadas deve estar na estrutura
```
dataset/
├── coxinha/
│   ├── coxinha0001.jpg
│   ├── coxinha0002.jpg
│   ├── coxinha0003.jpg
│   └── ...
├── bolinha_de_queijo/
│   ├── bolinha0001.jpg
│   ├── bolinha0002.jpg
│   └── ...
└── pastel/
    ├── pastel0001.jpg
    ├── pastel0002.jpg
    └── ...
```


In [ ]:
input_folder = "dataset"
output_folder = "dataset_sinth"

#coleta paths de todas as imagens
paths = collect_image_paths(input_folder) 

#remove da lista imagens do qual as vistas sintéticas já foram criadas
paths = filter_unprocessed(paths, input_folder, output_folder)

In [ ]:
#imprime os paths das imagens baixadas do google
paths

In [ ]:
# o cliente recebe a chave da API declarada acima
client = genai.Client(api_key=api_key)

#o prompt detalha as visadas da imagem apresentada que nos interessam.
prompt = ("""
Hi! I'm doing a image training dataset of Brazilian Salgadinhos, and want to generate novel views of the image i sent you. Follow the generation guide below please!
 {

  "style_mode": "raw_photoreal_documentary_collage",

  "look": "casual product photography, domestic setting, natural and ambient lighting, 2x2 grid layout",

  "layout_structure": {

    "format": "four-panel collage (2x2 grid)",

    "description": "The image is divided into four distinct rectangular quadrants, each presenting a different angle and lighting condition of the subject on the source image."

  },

  "camera": {

    "vantage": "variable per quadrant (high-angle, eye-level, low-angle, top-down)",

    "framing": "medium to close-up shots",

    "lens_behavior": "smartphone camera aesthetic, varying depth of field (shallow in macro shots, deep in environmental shots)",

    "sensor_quality": "standard digital photography, slight ISO noise in darker areas, realistic sharpness"

  },

  "scene": {

    "quadrant_details": {

      "top_left": {

        "perspective": "high-angle three-quarter view",

        "lighting": "same as the source image",

        "background": "same as the source image",

      },

      "top_right": {

        "perspective": "direct top-down (flat lay)",

        "lighting": "same as the source image",

        "background": "Salgadinhos on the image out of the tray and on the floor",


      },

      "bottom_left": {

        "perspective": "direct top-down (flat lay)",

        "lighting": "nightime lighting",

        "background": "same as the source image",

      },

      "bottom_right": {

        "perspective": "direct top-down (flat lay)",

        "lighting": "same as the source image",

        "background": "same as the source image",


      }

    }

  },

  "aesthetic_controls": {

    "render_intent": "view expansion of source image",

    "material_fidelity": [

    "same as source, upscale as needed."
    
    ],

    "color_grade": {

      "overall": "same as source)",


    }

  },

  "negative_prompt": {

    "forbidden_elements": [

      "people",

      "animals (living)",

      "bright neon colors",

      "text overlays",

      "studio backdrop",

      "vector graphics",

      "cartoon style"

    ]

  }

} 
""")

#processamos as imagens baixadas, gerando novas vistas e augumentando o dataset
process_paths(paths, input_folder, output_folder, client, prompt)

In [ ]:
#visualização das imagens geradas, salva como collage.jpg no diretório atual do notebook
create_collage("dataset_sinth/coxinha")

In [ ]:
#recortamos as imagens geradas em 4, salvando cada corte individualmente
slice_images_into_four('dataset_sinth','dataset_sinth_cut')

### *Labeling* bruto com sam3

Para treinar modelos de detecção de objetos capazes de identificar, classificar e contar elementos em imagens, é fundamental estruturar adequadamente o dataset de treinamento. 

Cada imagem precisa estar acompanhada de anotações que especifiquem a localização e categoria dos objetos presentes, sendo essas informações armazenadas em arquivos de labels. Existem diversos formatos consolidados na comunidade de visão computacional: 

- o **formato YOLO** utiliza arquivos `.txt` individuais para cada imagem, com coordenadas normalizadas no padrão `<classe> <x_centro> <y_centro> <largura> <altura>` (valores entre 0 e 1);
- o **formato COCO** (Common Objects in Context) emprega um único arquivo JSON centralizado contendo todas as anotações do dataset, com coordenadas absolutas em pixels no formato `[x_min, y_min, largura, altura]` e suporte nativo para segmentação e keypoints;
- o **formato Pascal VOC** (Visual Object Classes) usa arquivos XML individuais por imagem, armazenando bounding boxes como `<xmin>, <ymin>, <xmax>, <ymax>` em pixels absolutos.

A escolha entre esses formatos frequentemente depende das ferramentas de anotação disponíveis e do framework de treinamento, embora conversões entre formatos sejam relativamente simples. Independentemente do formato escolhido, a qualidade e consistência das anotações são fatores determinantes para o desempenho do modelo em tarefas de detecção e contagem de objetos.

Do [README do modelo](https://github.com/facebookresearch/sam3/blob/main/README.md):
> SAM 3 is a unified foundation model for promptable segmentation in images and videos. It can detect, segment, and track objects using text or visual prompts such as points, boxes, and masks. Compared to its predecessor SAM 2, SAM 3 introduces the ability to exhaustively segment all instances of an open-vocabulary concept specified by a short text phrase or exemplars.

Usamos o modelo sam3 para, com um prompt simples como 'food', 'snacks' ou 'croquette', segmentar e gerar bounding boxes para os salgadinhos presentes em uma batelada de imagens, separadas por classe. Salvamos esses resultados com .txts no formato YOLO. O conjunto de fotos de 'grupo' é salvo com a classe 'coxinha'. No proximo passo usaremos a ferramenta 'label studio' para consertar labelings equivocados.

![bounding boxes geradas por sam3](relatorio2Images/sam3_raw_bbox.png)
Acima: Bounding boxes geradas por sam3

#### *Labeling* bruto com sam3: código

##### bibiliotecas e instalações

In [ ]:
!pip install -U torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [1]:
#baixando e instalando sam3 a partir do github

import os
from pathlib import Path

# save current dir
start_dir = Path.cwd()

# clone
!git clone https://github.com/facebookresearch/sam3.git

# go into repo
os.chdir("sam3")

# install things
!pip install -e .
!pip install -e ".[notebooks]"

# return to original notebook dir
os.chdir(start_dir)

print("Now back in:", Path.cwd())


Cloning into 'sam3'...
remote: Enumerating objects: 578, done.
remote: Total 578 (delta 0), reused 0 (delta 0), pack-reused 578 (from 1)
Receiving objects: 100% (578/578), 58.92 MiB | 10.62 MiB/s, done.
Resolving deltas: 100% (70/70), done.
Now in: /media/vicente/lindata/pseudoHome/00CORE/UFSC/2025Mestrado/CompVis/cv-ine410121-2025/sam3
Now back in: /media/vicente/lindata/pseudoHome/00CORE/UFSC/2025Mestrado/CompVis/cv-ine410121-2025


##### imports e definições

In [ ]:
import os

import numpy as np

import sam3
from PIL import Image

import sys

#utils sam3
from huggingface_hub import login
from sam3.sam3 import build_sam3_image_model
from sam3.model.box_ops import box_xywh_to_cxcywh
from sam3.model.sam3_image_processor import Sam3Processor
from sam3.visualization_utils import normalize_bbox
import torch

#para mostrar as bounding boxes
import glob
import cv2
import random
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, Markdown


##### funções

In [ ]:
def strip_ext(path):
    # Remove a extensão do arquivo do caminho fornecido
    # Ex: "imagem.jpg" -> "imagem"
    return os.path.splitext(path)[0]

def convert_to_yolo(bbox, img_width, img_height):
    # Desempacota as coordenadas da bounding box (canto superior esquerdo e inferior direito)
    x_min, y_min, x_max, y_max = bbox
    
    # Calcular centro e dimensões
    # Calcula a coordenada X do centro da bounding box
    x_center = (x_min + x_max) / 2
    # Calcula a coordenada Y do centro da bounding box
    y_center = (y_min + y_max) / 2
    # Calcula a largura da bounding box
    width = x_max - x_min
    # Calcula a altura da bounding box
    height = y_max - y_min
    
    # Normalizar pelos tamanhos da imagem
    # Normaliza a coordenada X do centro dividindo pela largura da imagem (valor entre 0 e 1)
    x_center_norm = x_center / img_width
    # Normaliza a coordenada Y do centro dividindo pela altura da imagem (valor entre 0 e 1)
    y_center_norm = y_center / img_height
    # Normaliza a largura da bounding box dividindo pela largura da imagem (valor entre 0 e 1)
    width_norm = width / img_width
    # Normaliza a altura da bounding box dividindo pela altura da imagem (valor entre 0 e 1)
    height_norm = height / img_height
    
    # Retorna a bounding box no formato YOLO (centro_x, centro_y, largura, altura) normalizado
    return [x_center_norm, y_center_norm, width_norm, height_norm]

def SAM3_to_YOLO(image_path, processor, txt_prompt='snack', yolo_class=3):
    # Remove a extensão do caminho da imagem para obter o nome base
    image_name=strip_ext(image_path)
    # Cria o caminho do arquivo de texto de saída com o mesmo nome da imagem
    txt_path = f"{image_name}.txt" 
    
    # Abre a imagem usando PIL
    image = Image.open(image_path)
    # Obtém as dimensões (largura e altura) da imagem
    width, height = image.size
    # Configura a imagem no processador SAM e obtém o estado de inferência
    inference_state = processor.set_image(image)
    # Reseta todos os prompts anteriores no estado de inferência
    processor.reset_all_prompts(inference_state)
    # Define o prompt de texto para detecção e obtém os resultados
    results = processor.set_text_prompt(state=inference_state, prompt=txt_prompt)
    # Obtém o número de objetos detectados através da quantidade de scores retornados
    number_of_objects = len(results["scores"])
    # Inicializa string vazia para armazenar o conteúdo no formato YOLO
    yolo_content=''
    # Itera sobre cada objeto detectado
    for i in range(number_of_objects):
        # Obtém a bounding box do objeto i, converte para CPU e transforma em lista Python
        bounding_box=results["boxes"][i].cpu().tolist()
        # Converte a bounding box para o formato YOLO normalizado
        yolo_bbox = convert_to_yolo(bounding_box, width, height)
        # Define o ID da classe do objeto (fixo como yolo_class)
        class_id = yolo_class # ID da classe do objeto
        # Formata a linha no formato YOLO: "class_id x_center y_center width height"
        yolo_line = f"{class_id} {' '.join(map(str, yolo_bbox))}"
    
        # Adiciona a linha formatada ao conteúdo, seguida de quebra de linha
        yolo_content=yolo_content+yolo_line+'\n'
    
    # Abre o arquivo de texto em modo escrita
    with open(txt_path, "w") as f:
        # Escreve todo o conteúdo YOLO no arquivo
        f.write(yolo_content)
    # Retorna a quantidade de objetos detectados
    return number_of_objects

def parse_yolo_txt(txt_path, img_w, img_h):
    # Inicializa lista vazia para armazenar as bounding boxes
    boxes = []
    # Verifica se o arquivo de texto existe
    if not os.path.exists(txt_path):
        # Retorna lista vazia se o arquivo não existir
        return boxes
    # Abre o arquivo de texto em modo leitura
    with open(txt_path, "r") as f:
        # Itera sobre cada linha do arquivo
        for line in f:
            # Remove espaços em branco no início e fim da linha
            line = line.strip()
            # Pula linhas vazias
            if not line:
                continue
            # Divide a linha em partes separadas por espaço
            parts = line.split()
            # Verifica se a linha tem pelo menos 5 valores (classe + 4 coordenadas)
            if len(parts) < 5:
                continue
            # Extrai o ID da classe e converte para inteiro
            cls = int(float(parts[0]))
            # Extrai a coordenada X do centro normalizada
            x_c = float(parts[1])
            # Extrai a coordenada Y do centro normalizada
            y_c = float(parts[2])
            # Extrai a largura normalizada
            w = float(parts[3])
            # Extrai a altura normalizada
            h = float(parts[4])
            # Calcula a coordenada X do canto superior esquerdo em pixels
            x1 = int((x_c - w/2) * img_w)
            # Calcula a coordenada Y do canto superior esquerdo em pixels
            y1 = int((y_c - h/2) * img_h)
            # Calcula a coordenada X do canto inferior direito em pixels
            x2 = int((x_c + w/2) * img_w)
            # Calcula a coordenada Y do canto inferior direito em pixels
            y2 = int((y_c + h/2) * img_h)
            # Limita x1 e x2 aos limites da imagem (0 até largura-1)
            x1 = max(0, min(img_w-1, x1)); x2 = max(0, min(img_w-1, x2))
            # Limita y1 e y2 aos limites da imagem (0 até altura-1)
            y1 = max(0, min(img_h-1, y1)); y2 = max(0, min(img_h-1, y2))
            # Adiciona a tupla (classe, x1, y1, x2, y2) à lista de boxes
            boxes.append((cls, x1, y1, x2, y2))
    # Retorna a lista de bounding boxes
    return boxes

def preview_yolo_folder(folder, max_images=20, alpha=0.5, figsize=(12,8), show_filenames=True):
    # Converte o caminho da pasta para objeto Path
    folder = Path(folder)
    # Verifica se a pasta existe e se é realmente um diretório
    if not folder.exists() or not folder.is_dir():
        # Lança erro se a pasta não for encontrada
        raise ValueError(f"Folder not found: {folder}")
    # Define lista de extensões de imagem suportadas
    exts = ["*.jpg", "*.jpeg", "*.png", "*.bmp"]
    # Inicializa lista vazia para armazenar caminhos das imagens
    image_paths = []
    # Itera sobre cada extensão de arquivo
    for e in exts:
        # Busca todos os arquivos com a extensão atual e adiciona à lista (ordenados)
        image_paths.extend(sorted(folder.glob(e)))
    # Verifica se alguma imagem foi encontrada
    if not image_paths:
        # Lança erro se nenhuma imagem for encontrada
        raise ValueError(f"No images found in {folder}")
    # Limita o número de imagens ao máximo especificado
    image_paths = image_paths[:max_images]
    # Itera sobre cada caminho de imagem
    for img_path in image_paths:
        # Carrega a imagem usando OpenCV
        img = cv2.imread(str(img_path))
        # Verifica se a imagem foi carregada com sucesso
        if img is None:
            # Exibe mensagem de erro e pula para a próxima imagem
            print(f"Failed to load image: {img_path}")
            continue
        # Obtém altura e largura da imagem
        h, w = img.shape[:2]
        # Obtém o caminho do arquivo .txt correspondente (mesmo nome, extensão .txt)
        txt_path = img_path.with_suffix(".txt")
        # Faz o parse do arquivo YOLO para obter as bounding boxes
        boxes = parse_yolo_txt(txt_path, w, h)
        # Cria uma cópia da imagem para o overlay (camada de cor transparente)
        overlay = img.copy()
        # Inicializa dicionário para armazenar cores por classe
        colors = {}
        # Itera sobre cada bounding box detectada
        for box in boxes:
            # Desempacota os valores da box (classe e coordenadas)
            cls, x1, y1, x2, y2 = box
            # Verifica se já existe uma cor definida para essa classe
            if cls not in colors:
                # Cria um gerador de números aleatórios com seed baseado na classe
                rnd = random.Random(cls)
                # Gera uma cor RGB aleatória (valores entre 50 e 255)
                colors[cls] = (int(rnd.randint(50,255)), int(rnd.randint(50,255)), int(rnd.randint(50,255)))
            # Obtém a cor para a classe atual
            color = colors[cls]
            # Desenha um retângulo preenchido no overlay com a cor da classe
            cv2.rectangle(overlay, (x1,y1), (x2,y2), color, thickness=-1)
            # Desenha a borda preta do retângulo na imagem original
            cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,0), thickness=2)
            # Adiciona o texto com o ID da classe no canto superior esquerdo da box
            cv2.putText(img, str(cls), (x1+3, y1+18), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)
        # Mistura o overlay colorido com a imagem original usando o fator alpha
        blended = cv2.addWeighted(overlay, alpha, img, 1-alpha, 0)
        # Converte a imagem de BGR (OpenCV) para RGB (matplotlib)
        blended_rgb = cv2.cvtColor(blended, cv2.COLOR_BGR2RGB)
        # Verifica se deve mostrar os nomes dos arquivos
        if show_filenames:
            # Exibe o nome do arquivo e número de boxes em formato Markdown
            display(Markdown(f"**Preview:** `{img_path.name}`  —  {len(boxes)} box(es)"))
        # Cria uma nova figura com o tamanho especificado
        plt.figure(figsize=figsize)
        # Remove os eixos da visualização
        plt.axis('off')
        # Exibe a imagem mesclada
        plt.imshow(blended_rgb)
        # Renderiza a visualização
        plt.show()

##### execução

In [ ]:
# turn on tfloat32 for Ampere GPUs
# https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# use bfloat16 for the entire notebook
torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

###### construir o modelo
[visite esta pagina para requisitar acesso pelo Hugging Face](https://huggingface.co/facebook/sam3)

In [ ]:
#abrir token para a API da plataforma Hugging Face
with open('../hf_token.txt', 'r') as file:
    my_token = file.read()

#login
login(token=my_token)

#instância do modelo
bpe_path = f"./sam3/assets/bpe_simple_vocab_16e6.txt.gz"
model = build_sam3_image_model(bpe_path=bpe_path)
processor = Sam3Processor(model, confidence_threshold=0.5)

###### inferencia
esse processo foi realizado varias vezes por pasta de imagens da categoria, onde selecionamos a melhor palavra chave para o modelo encontrar o salgado nas imagens. Uma boa palavra para sanduiches não mapeia para coxinhas. "Food" e "Snack" foram os termos mais amplos identificados

In [ ]:

#selecionar classe para realizar o labeling em batelada
folder = "./dataset_sinth_cut/sanduiche"

salgados_e_classes={"Bolinha de queijo":0,"Canapé":1,"Canudo":2,"Coxinha":3,"Croquete":4,"Empadinha":5,"Enroladinho de salsicha":6,"Esfiha":7,"Folhado":8,"Pastelzinho":9,"Pão de queijo":10,"Quibe":11,"Risoles":12,"Sanduiche":13}

#selecionar o salgado da pasta, o dicionário vai mapear para o encoding YOLO
set_class = salgados_e_classes["Sanduiche"]

# valid image extensions
exts = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".webp"}

#habilita uma segunda tentativa com um termo mais amplo caso o modelo não crie nenhum label com o prompt principal
second_try=True

#file iterator
for root, _, files in os.walk(folder):
    for f in files:
        if os.path.splitext(f)[1].lower() in exts:
            path = os.path.join(root, f)
            total = SAM3_to_YOLO(path, 
                                 processor, 
                                 txt_prompt='sandwich', #main prompt
                                 yolo_class = set_class)
            if total == 0 and second_try==True:
                print('0 ->', end = ' ')
                total = SAM3_to_YOLO(path, 
                                 processor, 
                                 txt_prompt='hors d’oeuvres', #fallback prompt
                                 yolo_class = set_class)
            print(total)


In [ ]:
# preview the labels
folder = "./dataset_sinth_cut/sanduiche"
preview_yolo_folder(folder, max_images=40)

### Refinamento de labels com Label Studio